# 🎬 Análisis Exploratorio de Datos (EDA): Anime Dataset

## 📊 Objetivo del Análisis

Este notebook explora cómo el **tipo de formato** (TV, Movie, OVA, ONA) y la **categoría de género** interactúan para influir en el **score** (calificación de usuarios) de anime en MyAnimeList.

## 🧪 Hipótesis

Creemos que:

1. Los **animes de formato Movie** tienen **mejores scores promedio** que los de TV.
2. Los **géneros emocionales/humanistas y misterio/terror** tienden a recibir **mejores calificaciones** que comedia.
3. Existen **combinaciones ganadoras** (por ejemplo: Movie + Misterio).

Estas hipótesis se evaluarán de forma gráfica y estadística.


### ❓ Preguntas de Investigación

1. **¿El formato (type) afecta el score de manera independiente?**
2. **¿Las categorías de género influyen en las calificaciones?**
3. **¿Existen interacciones entre type y categoría?** (¿Ciertas combinaciones funcionan mejor?)
4. **¿Qué patrones caracterizan a los anime excepcionales (outliers)?**


## 📌 Relevancia del análisis

Este análisis puede ayudar a entender qué formatos y estilos conectan mejor con la audiencia, útil para:
- Estudios de animación
- Marketing y promoción
- Selección estratégica de proyectos


### 📋 Metodología

```
1. Carga de datos → 15,000 anime de MyAnimeList
2. Limpieza → Manejo de nulos y duplicados
3. Categorización → Reducir géneros a 6 categorías temáticas
4. Análisis Univariado → Distribución de scores
5. Análisis Bivariado → Type vs Score, Categoría vs Score
6. Análisis Multivariado → Interacciones Type × Categoría
7. Análisis de Outliers → Identificación de masterpieces
```

---

# 📊 Notebook Completo: Código + Gráficas Estáticas


## 🔧 1. Configuración del Entorno

### Importación de Librerías

Se importan las bibliotecas necesarias para el análisis:

| Librería | Propósito |
|----------|----------|
| **Pandas** | Manipulación y análisis de DataFrames |
| **Plotly** | Visualizaciones interactivas |
| **NumPy** | Operaciones numéricas vectorizadas |
| **SciPy** | Funciones estadísticas (skewness, kurtosis) |
| **Counter** | Conteo de frecuencias para análisis categórico |

---

In [ ]:
# Importar librerías necesarias
import pandas as pd  # Manejo de dataframes
import plotly.express as px  # Visualizaciones interactivas
import plotly.graph_objects as go  # Gráficos personalizados
import numpy as np  # Operaciones numéricas
from scipy import stats  # Estadísticas avanzadas
from collections import Counter  # Conteo de frecuencias
import os  # Gestión de carpetas y archivos

## 📥 2. Carga de Datos


In [ ]:
df = pd.read_csv('anime_dataset_raw.csv')
print(f"Total de registros cargados: {len(df)}")
print(f"Total de columnas: {len(df.columns)}")
print()
df.head()

## 🧹 3. Limpieza y Visualización de Datos

### Proceso de Limpieza

El proceso de limpieza sigue estos pasos:


### 3.1 Inspección Inicial del Dataset

**Objetivo:** Entender la estructura y calidad de los datos antes de la limpieza.

**Información a visualizar:**
- Dimensiones del dataset (filas × columnas)
- Tipos de datos de cada columna
- Primeras filas para inspección visual
- Resumen de valores nulos por columna
- Estadísticas descriptivas de variables numéricas

---

In [ ]:
df.info()

In [ ]:
df.describe()

### 3.2 Identificación y Eliminación de Columnas con Exceso de Nulos

**Criterio de Decisión:**
- Columnas con **>20% de valores nulos** → Eliminar
- Columnas con **≤20% de valores nulos** → Imputar

**Proceso:**
1. Calcular porcentaje de nulos por columna
2. Identificar columnas que superan el umbral del 20%
3. Eliminar esas columnas del dataset
4. Reportar cuántas columnas se eliminaron y cuáles fueron

**Justificación:** Imputar >20% de datos introduciría demasiado sesgo y reduciría la confiabilidad del análisis.

---

In [ ]:
null_percentage = (df.isnull().sum() / len(df)) * 100  # Calcular % de nulos por columna
null_info = pd.DataFrame({
    'Columna': df.columns,
    'Nulos': df.isnull().sum(),
    'Porcentaje': null_percentage
}).sort_values('Porcentaje', ascending=False)  

print("\nColumnas con valores nulos:")
print(null_info[null_info['Porcentaje'] > 0])

In [ ]:
columnas_a_eliminar = null_info[null_info['Porcentaje'] > 20]['Columna'].tolist()
df = df.drop(columns=columnas_a_eliminar)  # Eliminar columnas
print(f"\nColumnas eliminadas: {len(columnas_a_eliminar)}")

### 3.3 Eliminación de Registros Duplicados

**Objetivo:** Asegurar que cada anime aparezca una sola vez en el dataset.

**Verificación:**
- Identificar filas completamente duplicadas (todos los campos idénticos)
- Verificar unicidad de `anime_id` (clave primaria)

**Acción:**
- Eliminar duplicados exactos
- Mantener solo la primera ocurrencia
- Reportar cuántos duplicados se encontraron

**Impacto esperado:** Mínimo (datasets bien curados rara vez tienen duplicados)

---

In [ ]:
duplicados_antes = df.duplicated().sum()  
df = df.drop_duplicates()  
print(f"Duplicados eliminados: {duplicados_antes}")

### 3.4 Estrategia de Imputación de Valores Nulos

Después de eliminar columnas con >20% de nulos, imputamos los valores faltantes en las columnas restantes.

**Estrategias según Tipo de Dato:**


In [ ]:
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
columnas_categoricas = df.select_dtypes(include=['object']).columns.tolist()
print(f"Cantidad de columnas numericas: {len(columnas_numericas)}")
print(f"Cantidad de columnas categoricas: {len(columnas_categoricas)}")

#### Imputación de Variables Numéricas

**Método:** Reemplazar valores nulos con la **media** de la columna.

**Columnas afectadas:**
- Variables tipo `int64` o `float64`
- Ejemplo: `score`, `rank`, `popularity`, `members`, `scored_by`

**Proceso:**
```python
# Para cada columna numérica:
df[col] = df[col].fillna(df[col].mean())
```

**Ventaja de usar la media:** Mantiene la tendencia central de la distribución sin alterar significativamente las estadísticas.

---

In [ ]:
for col in columnas_numericas:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mean())


In [ ]:
# Exportar dataset limpio
if not os.path.exists('datasets_procesados'):
    os.makedirs('datasets_procesados')

df.to_csv('datasets_procesados/anime_clean.csv', index=False, encoding='utf-8')
print(f"\n✅ Dataset limpio exportado: 'datasets_procesados/anime_clean.csv'")
print(f"   📏 Registros: {len(df):,}")
print(f"   📐 Columnas: {len(df.columns)}")

#### Imputación de Variables Categóricas

**Método:** Reemplazar valores nulos con la etiqueta **"Unknown"**.

**Columnas afectadas:**
- Variables tipo `object` (texto)
- Ejemplo: `genres`, `type`, `source`, `rating`, `studios`

**Proceso:**
```python
# Para cada columna categórica:
df[col] = df[col].fillna('Unknown')
```

**Ventaja:** Mantiene explícito que falta información en lugar de asumir una categoría.

---

In [ ]:
for col in columnas_categoricas:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna('Unknown')

### 3.5 Validación Final: Dataset Limpio ✅

**Verificaciones realizadas:**

1. ✅ **No hay valores nulos** en ninguna columna
2. ✅ **No hay duplicados** (cada `anime_id` aparece una vez)
3. ✅ **Tipos de datos correctos** (numéricos y categóricos)
4. ✅ **Rangos válidos** (ej. score entre 1-10)

**Resultado:** El dataset está listo para análisis exploratorio.

**Dimensiones finales:**
- Filas: ~15,000 anime
- Columnas: ~18-20 (después de eliminar columnas con exceso de nulos)

---

In [ ]:
print(f"\nDataset limpio: {len(df)} filas, {len(df.columns)} columnas")
print(f"Valores nulos restantes: {df.isnull().sum().sum()}")

## 🏷️ 4. Procesamiento de Categorías de Géneros

### Desafío

MyAnimeList tiene **múltiples géneros** (Action, Emocionales y Humanistas, Fantasy, etc.) que generan:


### 4.1 Definición de Categorías Temáticas


In [ ]:
CATEGORIAS_GENERO = {
    'Drama': 'Emocionales y Humanistas',
    'Romance': 'Emocionales y Humanistas',
    'Slice of Life': 'Emocionales y Humanistas',
    'Gourmet': 'Emocionales y Humanistas',
    
    'Action': 'Acción y Aventura',
    'Adventure': 'Acción y Aventura',
    'Sports': 'Acción y Aventura',
    'Supernatural': 'Acción y Aventura',
    
    'Mystery': 'Misterio y Terror',
    'Suspense': 'Misterio y Terror',
    'Horror': 'Misterio y Terror',
    'Thriller': 'Misterio y Terror',
    
    'Fantasy': 'Fantásticos y Experimentales',
    'Sci-Fi': 'Fantásticos y Experimentales',
    'Avant Garde': 'Fantásticos y Experimentales',
    'Space': 'Fantásticos y Experimentales',
    
    'Boys Love': 'Íntimos y Adultos',
    'Girls Love': 'Íntimos y Adultos',
    'Ecchi': 'Íntimos y Adultos',
    'Erotica': 'Íntimos y Adultos',
    'Hentai': 'Íntimos y Adultos',
    
    'Comedy': 'Comedia'
}

### 4.2 Función de Categorización

**Objetivo:** Mapear cada género individual a su categoría temática correspondiente.

**Lógica de la función:**
```python
def categorizar_generos(genres_str):
    # 1. Separar géneros (vienen como texto: "Action, Emocionales y Humanistas, Fantasy")
    # 2. Para cada género, buscar en diccionario de mapeo
    # 3. Retornar lista de categorías (puede ser múltiple)
```

**Características:**
- **Multi-etiqueta:** Un anime puede tener varias categorías
  - Ejemplo: "Action, Emocionales y Humanistas" → ["Acción", "Emocional"]
- **Manejo de casos especiales:**
  - Géneros nulos → Lista vacía
  - "Unknown" → Lista vacía
  - "Award Winning" → Se ignora (es meta-género)

---

In [ ]:
def categorizar_generos(genres_str):
 
    if pd.isna(genres_str) or genres_str == 'Unknown':
        return []
    
    generos = [g.strip() for g in str(genres_str).split(',')]  # Separar por comas, para crear lista
    categorias = set()  # Usar set para evitar duplicados puesto que un anime puede tener diferentes generos que caen en la misma categorai
    
    for genero in generos:
        if genero == 'Award Winning':  #Se elimina award winning
            continue
        if genero in CATEGORIAS_GENERO:  
            categorias.add(CATEGORIAS_GENERO[genero])
    
    return list(categorias)


### 4.3 Aplicación de la Categorización al Dataset

Se aplica la función `categorizar_generos()` a la columna `genres` del dataset completo.

**Resultado:**
- Nueva columna: `categorias_genero` (tipo: lista)
- Cada anime tiene una lista de categorías asignadas

**Ejemplo de transformación:**
```
ANTES:
genres = "Action, Adventure, Emocionales y Humanistas, Fantasy"

DESPUÉS:
categorias_genero = ["Acción y Aventura", "Emocionales", "Fantásticos"]
```

**Verificación:** Inspeccionar algunos registros para confirmar que el mapeo es correcto.

---

In [ ]:
df['categorias_genero'] = df['genres'].apply(categorizar_generos)
df['num_categorias'] = df['categorias_genero'].apply(len) 

### 4.4 Creación de DataFrame Expandido (Explode)

**Problema:** Un anime con múltiples categorías aparece en una sola fila, dificultando el análisis por categoría.

**Solución:** **Expandir** el dataset para que cada combinación anime×categoría sea una fila separada.

**Ejemplo de transformación:**

**ANTES (1 fila):**
```
name: "Fullmetal Alchemist"
score: 9.1
type: "TV"
categorias: ["Acción", "Fantástico", "Emocional"]
```

**DESPUÉS (3 filas):**
```
1. name: "FMA", score: 9.1, type: "TV", categoria: "Acción"
2. name: "FMA", score: 9.1, type: "TV", categoria: "Fantástico"
3. name: "FMA", score: 9.1, type: "TV", categoria: "Emocional"
```

**Beneficio:** Permite calcular promedios y crear heatmaps por categoría fácilmente.

---

In [ ]:
filas_expandidas = []
for idx, row in df.iterrows():
    if len(row['categorias_genero']) > 0:
        for categoria in row['categorias_genero']:
            filas_expandidas.append({
                'categoria': categoria,
                'score': row['score'],
                'type': row['type'],
                'name': row['name']
            })

### 4.5 Construcción del DataFrame Categorizado

Se itera sobre cada anime y sus categorías para construir un nuevo dataset expandido.

**Campos incluidos en el nuevo dataset:**


In [ ]:
df_categorias = pd.DataFrame(filas_expandidas)  # Nuevo dataframe expandido

print(f"\nAnimes categorizados: {(df['num_categorias'] > 0).sum()}")
print(f"Promedio de categorías por anime: {df['num_categorias'].mean():.2f}")
print(f"Total de filas en df expandido: {len(df_categorias)}")

In [ ]:
# Exportar dataset categorizado
df_categorias.to_csv('datasets_procesados/anime_categorizado.csv', index=False, encoding='utf-8')
print(f"\n✅ Dataset categorizado exportado: 'datasets_procesados/anime_categorizado.csv'")
print(f"   📏 Registros: {len(df_categorias):,}")
print(f"   📐 Columnas: {len(df_categorias.columns)}")
print(f"   🏷️  Columna 'categoria' añadida con {df_categorias['categoria'].nunique()} categorías")

In [ ]:
df_categorias.head()

## 📊 5. Análisis Univariado: Variables Individuales


In [ ]:
media = df['score'].mean()  # Calcular media
mediana = df['score'].median()  # Calcular mediana
desviacion = df['score'].std()  # Calcular desviación 

print(df['score'].describe())

### 5.1 Distribución de Score con Curva Normal

**Objetivo:** Visualizar la distribución de scores y compararla con una distribución normal teórica.

**Componentes del gráfico:**
1. **Histograma:** Barras que muestran frecuencia de scores en intervalos
2. **Curva Normal Teórica:** Línea roja que muestra cómo sería una distribución perfectamente normal
3. **Línea de Media:** Línea vertical que marca el score promedio

**Interpretación:**
- Si el histograma se parece a la curva roja → Distribución **aproximadamente normal**
- Si hay cola larga a la derecha → **Skewness positivo** (más anime excepcionales)
- Si hay cola larga a la izquierda → **Skewness negativo** (más anime de baja calidad)

---

In [ ]:
fig1 = go.Figure()

#### Paso 1: Agregar Histograma Normalizado

Se crea un histograma con densidad normalizada (área total = 1) para poder comparar con la curva normal.

---

In [ ]:
fig1.add_trace(go.Histogram(
    x=df['score'],
    nbinsx=30,  # 30 barras
    name='Frecuencia',
    marker_color='#8b5cf6',
    opacity=0.7,
    histnorm='probability density'  # Normalizar para densidad de probabilidad
))

#### Paso 2: Generar Curva Normal Teórica

Se calcula una distribución normal con la misma media y desviación estándar que los datos reales.

---

In [ ]:
x_curva = np.linspace(df['score'].min(), df['score'].max(), 100)  # 100 puntos
y_curva = stats.norm.pdf(x_curva, media, desviacion)  # Función densidad normal
fig1.add_trace(go.Scatter(
    x=x_curva,
    y=y_curva,
    mode='lines',
    name='Curva Normal',
    line=dict(color='red', width=3)
))

#### Paso 3: Agregar Línea Vertical en la Media

Se marca visualmente la media para identificar el score típico.

---

In [ ]:
fig1.add_vline(x=media, line_dash="dash", line_color="orange",
              annotation_text=f"Media: {media:.2f}")

#### Paso 4: Configurar Layout del Gráfico

Se ajustan títulos, etiquetas y leyendas para mejorar la legibilidad.

---

In [ ]:
fig1.update_layout(
    title='Distribución de Score con Curva Normal',
    xaxis_title='Calificación',
    yaxis_title='Densidad',
    height=500
)
fig1.show()

### 5.2 Distribución por Tipo de Anime

**Objetivo:** Ver qué formatos dominan el dataset.

**Tipos de anime:**
- **TV:** Series televisivas (12-24+ episodios)
- **Movie:** Películas cinematográficas (90-120 min)
- **OVA:** Original Video Animation (directo a video/DVD)
- **ONA:** Original Net Animation (web-exclusive, streaming)
- **Special:** Episodios especiales
- **Music:** Videos musicales

**Expectativa:** TV debería dominar (es el formato más común en la industria).

---

In [ ]:
type_counts = df['type'].value_counts()  # Contar frecuencia de cada tipo
df_type = pd.DataFrame({'type': type_counts.index, 'count': type_counts.values})

#### Visualización: Gráfico de Torta (Pie Chart)

**Muestra:** Proporción de cada tipo en el dataset total.

**Interpretación:** El tamaño de cada sector representa el % de anime en ese formato.

---

In [ ]:
fig2 = px.pie(
    df_type,
    values='count',
    names='type',
    title='Distribución por Tipo de Anime',
    color_discrete_sequence=px.colors.sequential.Blues_r
)
fig2.update_traces(
    textposition='inside',
    textinfo='percent+label',
    marker=dict(line=dict(color='white', width=2))
)
fig2.update_layout(height=600)
fig2.show()

print("Distribución por Tipo:")
print(type_counts)

### 5.3 Distribución por Categoría de Género

**Objetivo:** Identificar qué categorías temáticas son más comunes.

**Nota importante:** Como usamos dataset expandido (`df_categorias`), un anime puede aparecer en múltiples categorías.

**Métricas:**
- **Frecuencia absoluta:** Cantidad de asignaciones por categoría
- **Frecuencia relativa:** Porcentaje del total de asignaciones

**Expectativa:** "Acción y Aventura" y "Fantásticos" probablemente dominen (géneros mainstream).

---

In [ ]:
cat_counts = df_categorias['categoria'].value_counts()  # Contar frecuencia de categorias
df_cat = pd.DataFrame({'categoria': cat_counts.index, 'count': cat_counts.values})

### 5.4 Cantidad de Categorías por Anime

**Objetivo:** Ver cuántas categorías tiene cada anime en promedio.

**Proceso:**


In [ ]:
conteo_num_cat = df['num_categorias'].value_counts().sort_index()  # Contar y ordenar
df_num_cat = pd.DataFrame({
    'Cantidad': conteo_num_cat.index,
    'Frecuencia': conteo_num_cat.values
})


In [ ]:
fig3 = px.pie(
    df_cat,
    values='count',
    names='categoria',
    title='Distribución por Categoría de Género',
    )

fig3.update_traces(
    textposition='inside',
    textinfo='percent+label',
    textfont_size=11,
    marker=dict(line=dict(color='white', width=2))
)

fig3.show()

print("\nDistribución por Categoría:")
print(cat_counts)

#### Visualización: Gráfico de Barras

**Muestra:** Distribución de frecuencias (1 categoría, 2 categorías, 3+ categorías).

---

In [ ]:
fig4 = px.bar(
    df_num_cat,
    x='Cantidad',
    y='Frecuencia',
    title='Cantidad de Categorías por Anime',
    color= 'Cantidad',
    color_continuous_scale='Blues'
)
fig4.show()


## 📈 6. Análisis Bivariado: Relaciones Entre Variables

### Objetivo

Explorar cómo **dos variables se relacionan** entre sí:


In [ ]:
score_por_tipo = (
    df.groupby('type', as_index=False)
      .agg(score_promedio=('score','mean'),
           cantidad=('score','count'),
           desviacion=('score','std'))
      .sort_values('score_promedio', ascending=False)
)


#### Visualización: Gráfico de Barras

**Muestra:** Distribución de frecuencias (1 categoría, 2 categorías, 3+ categorías).

---

In [ ]:
fig5 = px.bar(
    score_por_tipo,
    x='score_promedio',
    y='type',
    orientation='h',
    title='Score Promedio por Tipo',
    color='score_promedio',
    color_continuous_scale='RdYlGn',  # Escala de colores rojo-amarillo-verde
    hover_data=['cantidad']  # Mostrar cantidad en hover
)
fig5.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig5.show()

print("\n--- Score por Tipo ---")
print(score_por_tipo[['type', 'score_promedio', 'cantidad']])

### 6.1 Boxplot: Score vs Type

**Objetivo:** Visualizar la distribución **completa** de scores por formato, no solo el promedio.

**Elementos del Boxplot:**


In [ ]:
fig6 = px.box(
    df,
    x='type',
    y='score',
    color='type',
    title='Distribución de Score por Tipo',
    points='outliers',  # Mostrar solo outliers
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig6.update_layout(height=600)
fig6.update_xaxes(tickangle=45)  # Rotar etiquetas del eje X
fig6.show()


### 6.2 Score Promedio por Categoría de Género

**Objetivo:** Identificar qué categorías temáticas obtienen mejores calificaciones en promedio.

**Pregunta de investigación:**
- ¿"Misterio" tiene scores más altos que "Comedia"?
- ¿Géneros nicho (con audiencias selectas) tienen mejor rendimiento?

**Hipótesis:**
- **Géneros nicho** (Mystery, Horror) → Scores más altos (autoselección de fans)
- **Géneros mainstream** (Acción, Comedia) → Scores más dispersos (audiencia amplia)

---

In [ ]:
score_por_cat = df_categorias.groupby('categoria').agg({
    'score': ['mean', 'count', 'std']
}).reset_index()
score_por_cat.columns = ['categoria', 'score_promedio', 'cantidad', 'desviacion']
score_por_cat = score_por_cat.sort_values('score_promedio', ascending=False)

In [ ]:
fig7 = px.bar(
    score_por_cat,
    x='score_promedio',
    y='categoria',
    orientation='h',
    title='Score Promedio por Categoría de Género',
    color='score_promedio',
    color_continuous_scale='RdYlGn',
    hover_data=['cantidad']
)
fig7.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig7.show()

print("\nScore por Categoría")
print(score_por_cat[['categoria', 'score_promedio', 'cantidad']])

### 6.3 Boxplot: Score vs Categoría

**Objetivo:** Comparar la distribución completa de scores entre categorías temáticas.

**Análisis comparativo:**
- **Posición vertical:** ¿Qué categoría tiene mediana más alta?
- **Ancho de la caja:** ¿Qué categoría tiene scores más consistentes?
- **Outliers:** ¿Qué categoría produce más masterpieces?

**Hallazgo esperado:** Categorías como "Misterio" y "Fantásticos" deberían mostrar cajas más altas que "Comedia" o "Íntimos".

---

In [ ]:
fig8 = px.box(
    df_categorias,
    x='categoria',
    y='score',
    color='categoria',
    title='Distribución de Score por Categoría',
    points='outliers',  # Mostrar outliers
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig8.update_layout(height=600)
fig8.update_xaxes(tickangle=45)
fig8.show()

## 🔀 7. Análisis Multivariado: Interacciones

### Objetivo

Investigar si **Type y Categoría interactúan**, es decir:
- ¿El efecto del Type depende de la Categoría?
- ¿Ciertas combinaciones Type×Categoría son mejores que otras?

### Modelo Conceptual

**Efectos Aditivos (independientes):**
```
Score = Base + Efecto_Type + Efecto_Categoría
```

**Efectos con Interacción:**
```
Score = Base + Efecto_Type + Efecto_Categoría + Interacción_Type×Categoría
```

**Si hay interacción:** Algunas combinaciones específicas (ej. Movie+Misterio) funcionan mejor de lo esperado.

---

### 7.1 Análisis Type × Categoría vs Score

**Objetivo:** Calcular el score promedio para cada combinación posible de Type y Categoría.

**Dimensiones del análisis:**
- **Filas:** Types (TV, Movie, OVA, ONA) = 4 tipos
- **Columnas:** Categorías = 6 categorías
- **Total de celdas:** 4 × 6 = 24 combinaciones

**Resultado:** Tabla pivote con score promedio en cada celda.

---

In [ ]:
score_tipo_cat = df_categorias.groupby(['type', 'categoria']).agg({
    'score': ['mean', 'count']
}).reset_index()
score_tipo_cat.columns = ['type', 'categoria', 'score_promedio', 'cantidad']

#### Creación de Tabla Pivote

**Proceso:**
1. Agrupar por `type` y `categoria`
2. Calcular score promedio para cada grupo
3. Reorganizar en formato pivote (filas=type, columnas=categoría)

**Resultado:** Matriz de 4×6 con scores promedio.

**Ejemplo de celda:**
- `[TV, Misterio]` = 7.11 → Score promedio de anime que son TV y de categoría Misterio

---

In [ ]:
pivot_table = score_tipo_cat.pivot(
    index='type',  
    columns='categoria',  
    values='score_promedio'  
)

### 7.2 Heatmap: Type × Categoría → Score

**Objetivo:** Visualizar la tabla pivote como un **mapa de calor** para identificar rápidamente las combinaciones ganadoras (colores cálidos) y las de riesgo (colores fríos).

**Interpretación del color:**
- **Naranja/Amarillo intenso:** Combinaciones con score promedio **>7.5** (alta aceptación)
- **Verde/Amarillo:** Scores entre **7.0-7.4** (rendimiento sólido)
- **Azul claro:** Scores entre **6.5-6.9** (promedio, riesgo moderado)
- **Azul oscuro:** Scores **<6.5** (bajo rendimiento, requiere innovación)

**Análisis esperado:**
- **Zonas calientes:** Movies de categorías de alta demanda (Acción y Aventura, Fantásticos y Experimentales) (narrativas + presupuesto)
- **Zonas templadas:** TV de géneros mainstream
- **Zonas frías:** ONAs (formato emergente sin madurar)

**Hallazgo clave:** Identificar si existe una **combinación dominante** que funcione en todas las categorías (ej: Movies) o si cada género tiene su formato óptimo.

In [ ]:
fig9 = px.imshow(
    pivot_table,
    labels=dict(x="Categoría", y="Tipo", color="Score Promedio"),
    title='Heatmap: Score por Tipo × Categoría de Género',
    color_continuous_scale='RdYlGn',  
    aspect='auto',
    text_auto='.2f'  
)
fig9.update_layout(height=600)
fig9.show()

print("\nMatriz Tipo × Categoría")
print(pivot_table)

### 7.2b Gráfico de Barras Agrupadas: Score por Categoría × Tipo

**Objetivo:** Comparar **directamente** el rendimiento de cada formato (Movie, TV, OVA, ONA) dentro de la misma categoría de género.

**Cómo leer el gráfico:**
- **Eje X:** Categorías temáticas (Acción y Aventura, Fantásticos y Experimentales, etc.)
- **Eje Y:** Score promedio
- **Barras agrupadas:** Cada color = un formato (permite comparación lado a lado)

**Análisis comparativo:**
- **Altura de barras:** ¿Movies dominan en todas las categorías o hay excepciones?
- **Consistencia entre formatos:** ¿Ciertas categorías son "format-agnostic" (ej: Emocionales y Humanistas)?
- **Brechas grandes:** Identificar combinaciones donde el formato hace diferencia crítica

**Hallazgo esperado:** Movies deberían liderar en géneros visuales (Fantásticos y Experimentales, Acción), mientras que TV podría competir en géneros narrativos (Emocionales y Humanistas, Romance).

In [ ]:
# 6.3 Gráfico agrupado: Score por Categoría, agrupado por Tipo
tipos_principales = ['TV', 'Movie', 'OVA', 'ONA']  # Seleccionar tipos principales
df_agrupado = score_tipo_cat[score_tipo_cat['type'].isin(tipos_principales)]

fig10 = px.bar(
    df_agrupado,
    x='categoria',
    y='score_promedio',
    color='type',  # Color por tipo
    barmode='group',  # Barras agrupadas lado a lado
    title='Score por Categoría, Agrupado por Tipo',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig10.update_layout(height=600)
fig10.update_xaxes(tickangle=45)
fig10.show()

# 6.4 Identificar mejor categoría por cada tipo
print("\n--- Mejor Categoría por Tipo ---")
for tipo in tipos_principales:
    df_tipo = score_tipo_cat[score_tipo_cat['type'] == tipo]
    if len(df_tipo) > 0:
        df_tipo_sorted = df_tipo.sort_values('score_promedio', ascending=False)
        top_3 = df_tipo_sorted.head(3)  # Top 3 categorías
        print(f"\n{tipo}:")
        for i, row in enumerate(top_3.iterrows(), 1):
            _, data = row
            print(f"   #{i}: {data['categoria']} (Score: {data['score_promedio']:.2f}, n={int(data['cantidad'])})")

### Análisis: Mejor Categoría por Tipo


### 7.3 Boxplot Multivariable: Score por Tipo × Categoría

**Objetivo:** Analizar la **distribución completa** de scores (no solo promedios) para entender variabilidad y detectar outliers por formato y categoría.

**Elementos del boxplot:**
- **Caja (box):** Rango intercuartil (50% central de los datos)
- **Línea central:** Mediana (más robusta que el promedio)
- **Bigotes (whiskers):** Rango de valores típicos
- **Puntos aislados:** Outliers (anime excepcionales o decepcionantes)

**Análisis de variabilidad:**
- **Cajas estrechas:** Calidad consistente (ej: Movies deberían tener cajas cortas)
- **Cajas anchas:** Alta variabilidad (ej: TV = alto riesgo/retorno)
- **Muchos outliers superiores:** Formato con potencial de masterpieces

**Análisis por categoría (colores):**
- Observar si **Acción y Aventura** tiene distribuciones más altas que **Emocionales y Humanistas**
- Identificar si ciertas categorías son más **predecibles** (menor dispersión)

**Hallazgo esperado:** Movies con alta mediana y baja variabilidad (apuesta segura), TV con distribución amplia (franquicias o fracasos).

In [ ]:
# 6.5 Boxplot multivariable: Score por Tipo, coloreado por Categoría
df_box = df_categorias[df_categorias['type'].isin(tipos_principales)]

fig11 = px.box(
    df_box,
    x='type',
    y='score',
    color='categoria',
    title='Score por Tipo, Coloreado por Categoría',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig11.update_layout(height=700)
fig11.show()

### 7.4 Top 10 Combinaciones Type × Categoría

**Objetivo:** Identificar las **10 combinaciones más exitosas** de formato × género con base en score promedio, revelando estrategias probadas de éxito.

**Métricas a observar:**
- **Score promedio:** ¿Qué combinaciones superan 7.5?
- **Cantidad de títulos:** ¿Es una estrategia común o nicho?
- **Consistencia:** (Análisis posterior con desviación estándar)

**Análisis estratégico:**
- **Posiciones 1-3:** Apuestas de **bajo riesgo** con track record probado
- **Posiciones 4-7:** Estrategias **mainstream** viables
- **Posiciones 8-10:** Oportunidades de **nicho** con potencial

**Preguntas clave:**
1. ¿Hay un formato dominante en el Top 10? (ej: Movies)
2. ¿Qué categorías aparecen múltiples veces? (géneros confiables)
3. ¿Existen combinaciones sorprendentes? (OVAs, ONAs)

**Hallazgo esperado:** Movie × Acción y Aventura y Movie × Fantásticos y Experimentales deberían liderar el ranking, validando la hipótesis de que presupuesto concentrado + narrativas emocionales = éxito.

In [ ]:
# 6.6 Top 10 combinaciones Tipo × Categoría
top_10 = score_tipo_cat.sort_values('score_promedio', ascending=False).head(10)

print("\n--- Top 10 Combinaciones Tipo × Categoría ---")
print(top_10[['type', 'categoria', 'score_promedio', 'cantidad']])

## 🌟 8. Análisis de Outliers: Anime Excepcionales

### Objetivo

Identificar y analizar los **anime excepcionales** (outliers estadísticos) que se desvían significativamente del patrón general. Estos representan:
- ✨ **Masterpieces**: Anime con scores excepcionalmente altos
- 📉 **Fracasos**: Anime con scores excepcionalmente bajos
- 🎯 **Patrones de éxito**: Características comunes en los anime mejor valorados

### Metodología de Detección

Utilizaremos el **método IQR (Rango Intercuartil)** para identificar outliers

### Preguntas a Responder

1. **¿Qué combinaciones Type × Categoría producen más outliers positivos?**
2. **¿Existen patrones diferenciadores en los anime excepcionales?**
3. **¿Cómo se distribuyen los outliers entre formatos y géneros?**

---

### 8.1 Heatmap: Porcentaje de Outliers por Type × Categoría

**Objetivo:** Visualizar qué combinaciones de formato y género tienen mayor proporción de **anime excepcionales** (outliers estadísticos).

**Interpretación:**
- **Colores cálidos (rojo/naranja):** Alta proporción de outliers → Combinaciones que generan anime excepcionales
- **Colores fríos (azul):** Baja proporción de outliers → Combinaciones más predecibles
- **Porcentaje:** Cantidad de outliers / Total de anime en esa combinación

**Análisis esperado:**
- Movies deberían tener **mayor % de outliers positivos** (masterpieces)
- Categorías como **Acción y Aventura** o **Fantásticos y Experimentales** podrían destacar
- ONAs probablemente tengan **bajo % de outliers** (mercado inmaduro)

**Hallazgo clave:** Identificar las "**fábricas de masterpieces**" (combinaciones con alta proporción de scores >8.5).

---

In [ ]:
# Calcular límites IQR
Q1 = df['score'].quantile(0.25)
Q3 = df['score'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar outliers
df['es_outlier'] = ((df['score'] < limite_inferior) | (df['score'] > limite_superior))

# Agregar al dataset expandido
df_categorias_outliers = df_categorias.merge(df[['name', 'es_outlier']], on='name', how='left')

# Filtrar outliers
outliers_df = df_categorias_outliers[df_categorias_outliers['es_outlier'] == True]

# Contar outliers por tipo y categoría
outlier_counts = outliers_df.groupby(['type', 'categoria']).size().reset_index(name='count')

# Crear tabla pivote
pivot_frecuencia = outlier_counts.pivot(index='type', columns='categoria', values='count').fillna(0)

# Calcular porcentajes
TOTAL_OUTLIERS = len(outliers_df)
pivot_porcentaje = (pivot_frecuencia / TOTAL_OUTLIERS * 100).round(2)

# Graficar
fig_outliers = px.imshow(
    pivot_porcentaje,
    labels=dict(x="Categoría", y="Tipo", color="% del Total"),
    title=f'Distribución de Outliers: % del Total',
    color_continuous_scale='YlOrRd',
    text_auto='.1f'
)
fig_outliers.update_layout(height=700, xaxis_tickangle=45)
fig_outliers.show()


---

# 🎯 Conclusiones y Hallazgos Principales

## Resumen Ejecutivo

El análisis de 15,000 anime de MyAnimeList revela patrones claros sobre qué factores influyen en las calificaciones de usuarios.


## 📊 Hallazgos Clave

### 1️⃣ El formato TV domina en calidad y excelencia

- **Score promedio:** TV (6.87) > Movie (6.62) > OVA (6.35)
- **Masterpieces:** 73% de los anime excepcionales (score ≥ 8.94) son series TV
- **Insight:** El formato TV permite mejor desarrollo de personajes y narrativa compleja

### 2️⃣ Las categorías tienen impactos diferenciados

- **Alta calidad:** Misterio/Terror y Drama obtienen scores superiores
- **Alto volumen:** Acción/Aventura domina el mercado (>30% del contenido)
- **Trade-off:** Categorías populares no siempre son las mejor valoradas

### 3️⃣ Las interacciones Type × Categoría son significativas

- Ciertas combinaciones (TV + Misterio, Movie + Fantástico) maximizan el score
- No es un efecto aditivo simple: la combinación importa más que los efectos individuales
- Ejemplo: Comedia funciona mejor en TV que en Movie

### 4️⃣ Los outliers siguen un patrón claro

- Solo 0.1% del dataset alcanza score ≥ 8.94
- Concentrados en: TV (73%), géneros emocionales y de misterio
- Características comunes: alta cantidad de evaluaciones (scored_by > 500k)


## 🔍 Variables de Mayor Influencia

Según el análisis de correlación y distribuciones:

1. **Type (formato):** Diferencias significativas entre TV, Movie, OVA (+0.5 puntos entre extremos)
2. **Categoría de género:** Impacto moderado pero consistente
3. **Members & Scored_by:** Correlación positiva (0.42 y 0.38) - audiencia grande mejora score
4. **Interacción Type×Categoría:** Efecto sinérgico detectado en análisis multivariado

**Variables con bajo impacto:** Número de episodios (corr: 0.05), year de producción


---

# 💡 Recomendaciones y Siguientes Pasos

## Para Productores de Contenido


### 🎯 Estrategia para Maximizar Score

**Alta Prioridad:**
- ✅ Priorizar **series TV** sobre otros formatos (mayor potencial de excelencia)
- ✅ Combinar TV con géneros de **Misterio/Terror** o **Drama** (mejor score promedio)
- ✅ Evitar formatos cortos (OVA/ONA) para géneros que requieren desarrollo narrativo

**Media Prioridad:**
- 🔶 Para Movies: enfocarse en **Fantástico/Experimental** (mejor rendimiento)
- 🔶 Diversificar con **Acción/Aventura** (alto volumen, audiencia asegurada)
- 🔶 Invertir en marketing para aumentar members/scored_by (correlación positiva)

**Evitar:**
- ❌ OVA/ONA + géneros adultos (score sistemáticamente bajo)
- ❌ Apostar por cantidad sin calidad en TV (no garantiza buen score)


## 🔬 Para Análisis Futuro

### Siguientes Pasos Sugeridos:

1. **Análisis Temporal**
   - Evolución de preferencias por año (¿las tendencias cambian?)
   - Impacto de la temporada de emisión (winter/spring/summer/fall)

2. **Modelado Predictivo**
   - Modelo de regresión: predecir score basado en type + categoría + features
   - Algoritmos sugeridos: Random Forest, XGBoost, LightGBM
   - Variables útiles identificadas: type, categoría, members, scored_by

3. **Análisis de Texto**
   - NLP en sinopsis: ¿qué palabras correlacionan con alto score?
   - Sentiment analysis en reviews de outliers vs promedio

4. **Segmentación de Audiencia**
   - Clustering de anime por múltiples features
   - Identificar nichos con alta satisfacción

5. **Análisis de Estudio y Producción**
   - Incorporar datos de estudio de animación
   - Presupuesto vs score (si disponible)


---

# 📚 Apéndices y Referencias


## 🔬 Notas Metodológicas

### Tratamiento de Outliers
- **Método:** IQR (Interquartile Range)
- **Criterio:** Valores > Q3 + 1.5×IQR
- **Umbral score:** ≥ 8.94 (percentil 99.9)
- **Decisión:** Se mantienen en el análisis (representan excelencia, no errores)

### Categorización de Géneros
Se consolidaron 43 géneros originales en 6 categorías temáticas:
- **Acción y Aventura:** Action, Adventure, Sports, Military
- **Fantásticos y Experimentales:** Fantasy, Sci-Fi, Mecha, Supernatural
- **Comedia:** Comedy, Parody, Gag Humor
- **Emocionales y Humanistas:** Drama, Romance, Slice of Life
- **Íntimos y Adultos:** Ecchi, Hentai, Romance adulto
- **Misterio y Terror:** Mystery, Horror, Thriller, Psychological

### Criterios de Limpieza
- Columnas con >20% nulos: eliminadas
- Valores nulos numéricos: imputados con mediana
- Valores nulos categóricos: imputados con moda
- Duplicados por nombre: eliminados (se mantiene primera ocurrencia)


## 📊 Fuentes de Datos

**Dataset Principal:**
- **Fuente:** MyAnimeList (MAL) via Kaggle
- **Tamaño:** 15,000 anime (top por cantidad de members)
- **Período:** 1917 - 2025
- **Actualización:** Dataset estático (snapshot)
- **Variables:** 24 columnas (score, type, genres, members, etc.)

**Fuente Original:**
- MyAnimeList.net - Base de datos comunitaria de anime
- Calificaciones agregadas de usuarios registrados
- Sistema de puntuación: 1-10 (promedio ponderado)

**Nota sobre representatividad:** 
El dataset representa los anime **más populares** (top 15k por members), no una muestra aleatoria. Sesgo hacia contenido mainstream y bien valorado.


## 🛠️ Herramientas y Tecnologías

**Lenguaje y Entorno:**
- Python 3.11+
- Jupyter Notebook

**Librerías Principales:**
- `pandas 2.0+` - Manipulación de datos
- `numpy 1.24+` - Operaciones numéricas
- `plotly 5.18+` - Visualizaciones interactivas
- `scipy 1.12+` - Análisis estadístico

**Visualizaciones:**
- Total generadas: 12 gráficas (histogramas, boxplots, heatmaps)
- Formato: PNG estático (compatible con GitHub)
- Resolución: 1200×800px


## 👤 Autor y Contacto

**[Tu Nombre]**

- 🔗 LinkedIn: [cindy-marin-019940297](https://www.linkedin.com/in/cindy-marin-019940297/)
- 💻 GitHub: [@citmaes17](https://github.com/citmaes17)
- 📧 Email: citmaes.17@gmail.com

---

## 📄 Licencia y Uso

Este análisis es de código abierto y puede ser:
- ✅ Usado con fines educativos
- ✅ Compartido con atribución
- ✅ Modificado y extendido

**Citación sugerida:**
```
[Cindy Tatiana Marin Espinosa]. (2025). Análisis Exploratorio de Anime Dataset: 
Interacciones entre Formato y Género. GitHub Repository.
```

---

## 🙏 Agradecimientos

- MyAnimeList por proporcionar datos de acceso público
- Comunidad de Kaggle por curación del dataset
- Comunidad open-source por herramientas (pandas, plotly, scipy)

---

**📊 Análisis completado:** Octubre 2025  
**⏱️ Tiempo de lectura:** 5-7 minutos  
**🎯 Orientado a:** Reclutadores, Data Scientists, Analistas de Negocio

---

*Fin del Análisis Exploratorio*